# Supermarket Transaction Analysis in SQL

The checkout line is a familiar part of the retail experience. Any retailer needs to understand their checkout data to answer important questions like "do people spend more with card or cash?", "how many items do people buy at different times?", and "are self-service checkouts quicker than those with a cashier?". 

In this notebook, we'll use SQL to analyze [transaction data from a Polish Supermarket](https://www.mdpi.com/2306-5729/4/2/67/htm) to answer financial and logistical questions.

A financial analysis of point of sale transactions can have many purposes. A non-exhaustive list is:
- increase sales by identifying what matters to the customer, is it promotions? on what items? is it opening hours? prices of particular items? and so on;
- increasing productivity of stores and operators by identifying factors that result in Lower processing time per item or basket size, accurate predictions for labour scheduling, adequate level of inventory, fewer scanning errors at checkout, and so on;
- predict sales in the short, medium and long term to inform decisions on investments in fixed assets and human resources, financing, go-to-market invesments, etc.

## Task 1: Exploring the data

A good first step in exploring a database is to select the first few rows from each table.

### Instructions

Select all columns and the first 100 rows from `pos_operator_logs` and `pos_transactions`.

In [1]:
SELECT * FROM 'pos_operator_logs.csv' LIMIT 10;

,id,Workstation_Group_ID,Workstation_ID,begin_date_time,operator_id
0,1,8,19,2019-02-13 05:37:55+00:00,269
1,2,8,18,2019-02-13 05:37:55+00:00,268
2,3,8,17,2019-02-13 05:38:43+00:00,267
3,4,1,4,2019-02-13 07:01:26+00:00,332
4,5,1,7,2019-02-13 07:01:57+00:00,10
5,6,1,7,2019-02-13 07:02:16+00:00,10
6,7,1,4,2019-02-13 07:03:54+00:00,332
7,8,1,4,2019-02-13 07:05:52+00:00,332
8,9,1,7,2019-02-13 07:32:51+00:00,114
9,10,1,7,2019-02-13 07:35:12+00:00,114


In [2]:
SELECT * FROM 'pos_transactions.csv' LIMIT 10;

,id,WorkstationGroupID,begin_date_time,end_date_time,OperatorID,basket_size,t_cash,t_card,amount
0,1,1,2017-12-07 14:23:23+00:00,2017-12-07 14:24:36+00:00,101,23,True,False,112.71
1,2,1,2017-12-07 14:25:09+00:00,2017-12-07 14:27:00+00:00,101,29,True,False,54.76
2,3,1,2017-12-07 14:27:28+00:00,2017-12-07 14:27:48+00:00,101,3,True,False,14.77
3,4,1,2017-12-07 14:28:04+00:00,2017-12-07 14:28:29+00:00,101,12,True,False,37.88
4,5,1,2017-12-07 14:29:40+00:00,2017-12-07 14:30:32+00:00,101,7,True,False,115.34
5,6,1,2017-12-07 14:31:03+00:00,2017-12-07 14:32:01+00:00,101,12,False,True,232.89
6,7,1,2017-12-07 14:32:25+00:00,2017-12-07 14:36:03+00:00,101,36,True,False,183.15
7,8,1,2017-12-07 14:36:32+00:00,2017-12-07 14:37:15+00:00,101,10,True,False,39.68
8,9,1,2017-12-07 14:37:31+00:00,2017-12-07 14:38:07+00:00,101,15,True,False,44.53
9,10,1,2017-12-07 14:38:21+00:00,2017-12-07 14:39:12+00:00,101,15,False,True,42.73


## Task 2: Do more people make transactions by card or by cash?

Understanding how people pay is crucial for shop logistics like deciding what kind of checkout equipment to buy.

### Instructions

Count how many transactions were processed by cash (the case when `t_cash` is true) and by card  (the case when `t_card` is true). Use the `pos_transactions` table.

In [5]:
SELECT 
	COUNT(CASE WHEN t_cash = 'True' THEN 1 END) AS cash_transactions, 
	COUNT(CASE WHEN t_card = 'True' THEN 1 END) AS card_transactions
FROM 'pos_transactions.csv';

,cash_transactions,card_transactions
0,84487,78246


## Task 3: Do people spend more per transaction when using cash or card?

Similarly, knowing how much people spend per transaction using different payment methods is helpful for deciding on security arrangments, and marketing to encourage shoppers to use different payment methods.

Find the average per transaction type when a transaction is only card, or only cash.

### Instructions

Calculate the mean (average) amount spent per transaction for cash and for card.

In [8]:
SELECT 
	ROUND(AVG(CASE WHEN t_cash = 'True' THEN amount END), 2) AS avg_cash_transactions, 
	ROUND(AVG(CASE WHEN t_card = 'True' THEN amount END), 2) AS avg_card_transactions
FROM 'pos_transactions.csv';

,avg_cash_transactions,avg_card_transactions
0,59.62,87.56


## Task 4: Exploring Sunday Trading

Regulations were introduced in Poland in 2018 banned shopping on some Sundays. It was generally two Sundays per month in 2018 and three Sundays per month in 2019. Supermarkets have reacted by extending the working hours on Fridays and Saturdays, while customers had to adapt to the changes in opening hours. **The dataset contains two working Sundays (24 February, 31 March) and two non-working Sundays (17 February, 7 April).**


### Instructions

Get the week number of the dates where trading occured on a Sunday. 

- Find the week number of `end_date_time` from `pos_transactions` for these Sundays: 17 February, 24 February,31 March and 7 April 2019.
- Order the dates from latest to earliest.

In [10]:
SELECT 
	DATE_PART('week', end_date_time::DATE) AS week_num,
	end_date_time::DATE AS end_date
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) >= 2019 
	AND end_date IN ('2019-02-17', '2019-02-24', '2019-03-31', '2019-04-07')
GROUP BY end_date
ORDER BY end_date;

,week_num,end_date
0,8,2019-02-24 00:00:00+00:00
1,13,2019-03-31 00:00:00+00:00


The last two dates in my query do not appear and rightly so, because they were non-working Sundays. 

So the Weeks with with working Sundays are 8 and 13 ("WS"). And the weeks with non-working Sundays are 7 and 14 ("NWS"). 

Let's explore the data further to see if I have the full week data. 

## Task 5: Analysis of daily trends

Understanding which days are busiest is important for deciding how many staff to schedule shifts for, as well as helping to plan financially.

### Instructions

- Select all transactions of 2019 and find out the daily sales amount, the daily average amount per transaction and the median average amount per transaction. 
- group by week

Do we see different trends on different days of the week? 

In [15]:
SELECT 
	DATE_PART('weekday', end_date_time::DATE) as week_day,
	ROUND(SUM(amount), 2) AS total_sales,
	ROUND(AVG(amount), 2) AS avg_sales_amount,
	PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount) AS median_sales_amount
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) = 2019 
GROUP BY week_day;

,week_day,total_sales,avg_sales_amount,median_sales_amount
0,0,312723.89,68.25,41.530
1,1,791646.02,59.08,37.715
2,2,736855.84,58.26,37.310
3,3,786957.71,61.48,38.390
4,4,1262586.61,71.20,47.475
5,5,1352373.49,78.73,51.150
6,6,1559677.83,86.33,56.750


Build on your analysis adding the average and median `basket_size` per day in 2019. 



In [16]:
SELECT 
	DATE_PART('weekday', end_date_time::DATE) as week_day,
	ROUND(AVG(basket_size), 2) AS avg_basket_size,
	PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY basket_size) AS median_basket_size
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) = 2019 
GROUP BY week_day;

,week_day,avg_basket_size,median_basket_size
0,0,15.95,10.0
1,1,14.13,10.0
2,2,14.17,10.0
3,3,14.07,10.0
4,4,16.07,11.0
5,5,18.03,12.0
6,6,19.91,13.0


## Task 6: Should the supermarket open on Sundays?

The two Sundays I see are significantly lower in volume of transactions than all other days. 
Also, Thursdays, Fridays and Saturdays are approximately 25% higher in number of transactions and 40% higher in sales revenue than Mondays, Tuesdays and Wednesdays.

Does it make sense to open stores on Sundays? 

The full week data is available only for week 8 (WS) and week 14 (NWS). Let's compare these two.


### Instructions

Calculate the total weekly sales amount for week 8 and 14 in 2019.

In [19]:
SELECT 
	DATE_PART('week', end_date_time::DATE) AS week_num,
	ROUND(SUM(amount), 2) AS total_sales
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) = 2019 
	AND week_num IN (8, 14)
GROUP BY week_num
ORDER BY week_num;

,week_num,total_sales
0,8,1686485.72
1,14,1690989.87


## Task 7: How much does labor cost?

Week 8 (WS) revenue is PLN1.7M and also week 14 (NWS) has PLN1.7M. It may be possible that customers go shopping before Sunday knowing that the store will be closed on that day. 

Assuming that the competition is going to also notice this trend and close on Sundays, and assuming that our stores can rearrange the workforce shifts to 6 days a week cutting the Sunday shifts **there may be possible savings from closing the stores on Sundays with no impact on revenue**. 

I start quantifying the savings with labour costs, how many operators were working on working Sundays? 

### Instructions

Find out how many operators were working on each day in 2019 by counting `operator_id `working on each `being_date_time`  

In [21]:
SELECT 
	DATE_PART('weekday', begin_date_time::DATE) as week_day,
	COUNT(operator_id) AS operator_num
FROM 'pos_operator_logs.csv' 
WHERE EXTRACT(YEAR FROM begin_date_time) = 2019
GROUP BY week_day
ORDER BY week_day;

,week_day,operator_num
0,0,574
1,1,1438
2,2,1502
3,3,1462
4,4,1507
5,5,1559
6,6,1354


The stores had 18 employees working on Sunday 24 February and 20 employees working on Sunday 31 March. 
I would then look into the hours per shift and the cost per hour to calcute the average savings per Sunday, as well as estimate all other costs of keeping stores open and operational (e.g. energy costs). On the other hand stores may have additional costs for not opening one day, for example for short shelf life goods that may go unsold. 